In [1]:
import pandas as pd
import locale
import glob
import os.path
import requests
import tarfile
import numpy as np
from keras.layers import Input, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten, LSTM, Reshape, Lambda
from keras.layers import Embedding
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model, Sequential
from keras import backend as K
from keras.layers  import TimeDistributed, TimeDistributedDense
import os, sys, re
from nltk import tokenize
import nltk
nltk.download('punkt')
import string 
import numpy as np

[nltk_data] Downloading package punkt to /home/qx344/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Using TensorFlow backend.


In [2]:



# Convert text to lower-case and strip punctuation/symbols from words
def normalize_text(text):
    norm_text = text.lower()

    # Replace breaks with spaces
    norm_text = norm_text.replace('<br />', ' ')

    # Pad punctuation with spaces on both sides
    for char in ['.', '"', ',', '(', ')', '!', '?', ';', ':']:
        norm_text = norm_text.replace(char, ' ' + char + ' ')

    return norm_text


In [3]:
fields = ['foldername', 'label']


df = pd.read_csv('/scratch/qx344/speech_data/presidents_meta.csv', usecols=fields,skipinitialspace=True)
# See the keys
print df.keys(),len(df.index)

Index([u'foldername', u'label'], dtype='object') 34


In [4]:
import gensim
from gensim.models.doc2vec import TaggedDocument
from collections import namedtuple
import io,glob,os, random

SentimentDocument = namedtuple('SentimentDocument', 'words tags split sentiment')

texts_test = [] 
texts_train = []  # list of text samples
# labels_index = {}  # dictionary mapping label name to numeric id
labels_train = []  # list of label ids
labels_test = []

#for line_no, line in enumerate(alldata):
index = 0
for line_no, row in df.iterrows():
    #sentiment = row['label']*1.0
    sentiment = 1 if row['label']>=4 else 0 
    os.chdir("/scratch/qx344/speech_data/data/{0}/Vectors".format(row['foldername']))
    filesNum = len(os.listdir('.'))
    testfile = random.choice(os.listdir('.'))
    readtxt = np.loadtxt(testfile)
    texts_test.append(readtxt)
    labels_test.append(row['label'])
    trainfilelist = glob.glob("*.txt")
    trainfilelist.remove(testfile)
    for txtfile in trainfilelist:
        readtxt = np.loadtxt(txtfile)
        index = index+1
        print index
        texts_train.append(readtxt)
        labels_train.append(row['label'])



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


/share/apps/numpy/1.11.1/intel/lib/python2.7/site-packages/numpy-1.11.1-py2.7-linux-x86_64.egg/numpy/lib/npyio.py:893: UserWarning: loadtxt: Empty input file: "speech-4563.txt"
  warnings.warn('loadtxt: Empty input file: "%s"' % fname)
/share/apps/numpy/1.11.1/intel/lib/python2.7/site-packages/numpy-1.11.1-py2.7-linux-x86_64.egg/numpy/lib/npyio.py:893: UserWarning: loadtxt: Empty input file: "speech-3793.txt"
  warnings.warn('loadtxt: Empty input file: "%s"' % fname)



323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572

/share/apps/numpy/1.11.1/intel/lib/python2.7/site-packages/numpy-1.11.1-py2.7-linux-x86_64.egg/numpy/lib/npyio.py:893: UserWarning: loadtxt: Empty input file: "address-to-congress-on-the-american-jobs-act.txt"
  warnings.warn('loadtxt: Empty input file: "%s"' % fname)
/share/apps/numpy/1.11.1/intel/lib/python2.7/site-packages/numpy-1.11.1-py2.7-linux-x86_64.egg/numpy/lib/npyio.py:893: UserWarning: loadtxt: Empty input file: "remarks-on-the-afghanistan-pullout.txt"
  warnings.warn('loadtxt: Empty input file: "%s"' % fname)



710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828


NameError: global name 'pad_dense_list' is not defined

In [6]:
data_train =  map(lambda x: pad_dense_list(x, 100),texts_train)
data_test =  map(lambda x: pad_dense_list(x, 100),texts_test)
labels_train = np.array(labels_train)
labels_test = np.array(labels_test)
data_train = np.array(data_train)
data_test= np.array(data_test)

In [7]:
data_train.shape

(828, 100, 4800)

In [5]:
def pad_dense_list(speech, maxsentence):
    
    each_sub_len = 4800#len(speech[0])
   
    Z = np.zeros((maxsentence, each_sub_len))
    #print("Shape of Z{}, list {}", Z.shape, len(speech))
    for index, row in enumerate(speech):
        if (index >= maxsentence):
            break
        
        Z[index] = row
    #print(Z) 
    return Z

def preprocess_text(texts):
    
    tokenizer = Tokenizer(nb_words=5)
    tokenizer.fit_on_texts(texts)
    text_encoded = np.zeros((len(texts),MAX_SENTENCES,MAX_SEQUENCE_LENGTH))

    word_index = tokenizer.word_index
    
    for enu, speech in enumerate(texts):
        sents = tokenize.sent_tokenize(speech)
        if(len(sents) == 0):
            continue
        sequences = tokenizer.texts_to_sequences(sents)
        
#         print(len(sequences))
#         print('Found %s unique tokens.' % len(word_index))
        data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
        text_encoded[enu] = (pad_dense_list(data, MAX_SENTENCES))
        
    return word_index, text_encoded

In [14]:
data =  map(lambda x: pad_dense_list(x, 100),texts)

In [16]:
data = np.array(data)

In [17]:
data.shape

(16, 100, 4800)

In [8]:
##replace 20 with number_of_speeches
##replace 64 with dimension of sentence vector (4800)
### Hierarchical+skip-thoughts


model = Sequential()

model.add(Reshape((1, 100, 4800), input_shape=(100, 4800)))
model.add(TimeDistributed(LSTM(128, input_shape=(100, 4800), return_sequences=True), input_shape=(100, 4800)))
model.add(Reshape((100,128), input_shape=(1,100,128)))
model.add(Lambda(lambda x: K.mean(x, axis=1, keepdims=True), output_shape=lambda s: (s[0], s[2]), input_shape=(100,128)))
model.add(Reshape((128,)))
model.add(Dense(7, input_dim=128, activation='softmax'))


model.compile(loss='sparse_categorical_crossentropy',optimizer='rmsprop',metrics=['acc'])

print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
reshape_1 (Reshape)              (None, 1, 100, 4800)  0           reshape_input_1[0][0]            
____________________________________________________________________________________________________
timedistributed_1 (TimeDistribute(None, 1, 100, 128)   2523648     reshape_1[0][0]                  
____________________________________________________________________________________________________
reshape_2 (Reshape)              (None, 100, 128)      0           timedistributed_1[0][0]          
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 128)           0           reshape_2[0][0]                  
___________________________________________________________________________________________

In [9]:
model.fit(data_train, labels_train, validation_data=(data_test, labels_test),nb_epoch=20, batch_size=10)

Train on 828 samples, validate on 34 samples
Epoch 1/20
828/828 [==============================] - 18s - loss: 1.6312 - acc: 0.2899 - val_loss: 1.4353 - val_acc: 0.4412
Epoch 2/20
828/828 [==============================] - 17s - loss: 1.4523 - acc: 0.3816 - val_loss: 1.3924 - val_acc: 0.5000
Epoch 3/20
828/828 [==============================] - 17s - loss: 1.3786 - acc: 0.4155 - val_loss: 1.3663 - val_acc: 0.4412
Epoch 4/20
828/828 [==============================] - 17s - loss: 1.3053 - acc: 0.4300 - val_loss: 1.2673 - val_acc: 0.5000
Epoch 5/20
828/828 [==============================] - 17s - loss: 1.3099 - acc: 0.4577 - val_loss: 1.3571 - val_acc: 0.4706
Epoch 6/20
828/828 [==============================] - 17s - loss: 1.2723 - acc: 0.4795 - val_loss: 1.3285 - val_acc: 0.5000
Epoch 7/20
828/828 [==============================] - 17s - loss: 1.1959 - acc: 0.4867 - val_loss: 1.2204 - val_acc: 0.5588
Epoch 8/20
828/828 [==============================] - 17s - loss: 1.1653 - acc: 0.5157 

In [12]:
##replace 20 with number_of_speeches
##replace 64 with dimension of sentence vector (4800)
### Hierarchical+skip-thoughts


model = Sequential()

model.add(Reshape((1, 100, 4800), input_shape=(100, 4800)))
model.add(TimeDistributed(LSTM(512, input_shape=(100, 4800), return_sequences=True), input_shape=(100, 4800)))
model.add(Reshape((100,512), input_shape=(1,100,512)))
model.add(Lambda(lambda x: K.mean(x, axis=1, keepdims=True), output_shape=lambda s: (s[0], s[2]), input_shape=(100,512)))
model.add(Reshape((512,)))
model.add(Dense(7, input_dim=512, activation='softmax'))


model.compile(loss='sparse_categorical_crossentropy',optimizer='rmsprop',metrics=['acc'])

print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
reshape_8 (Reshape)              (None, 1, 100, 4800)  0           reshape_input_4[0][0]            
____________________________________________________________________________________________________
timedistributed_4 (TimeDistribute(None, 1, 100, 512)   10881024    reshape_8[0][0]                  
____________________________________________________________________________________________________
reshape_9 (Reshape)              (None, 100, 512)      0           timedistributed_4[0][0]          
____________________________________________________________________________________________________
lambda_2 (Lambda)                (None, 512)           0           reshape_9[0][0]                  
___________________________________________________________________________________________

In [13]:
model.fit(data_train, labels_train, validation_data=(data_test, labels_test),nb_epoch=20, batch_size=10)

Train on 828 samples, validate on 34 samples
Epoch 1/20
828/828 [==============================] - 64s - loss: 1.6929 - acc: 0.2597 - val_loss: 1.5176 - val_acc: 0.4118
Epoch 2/20
828/828 [==============================] - 62s - loss: 1.4816 - acc: 0.3696 - val_loss: 1.3520 - val_acc: 0.5294
Epoch 3/20
828/828 [==============================] - 63s - loss: 1.4080 - acc: 0.4143 - val_loss: 1.3670 - val_acc: 0.5000
Epoch 4/20
828/828 [==============================] - 62s - loss: 1.3395 - acc: 0.4469 - val_loss: 1.2261 - val_acc: 0.5882
Epoch 5/20
828/828 [==============================] - 62s - loss: 1.2868 - acc: 0.4505 - val_loss: 1.2167 - val_acc: 0.5294
Epoch 6/20
828/828 [==============================] - 62s - loss: 1.2322 - acc: 0.4867 - val_loss: 1.1616 - val_acc: 0.5294
Epoch 7/20
828/828 [==============================] - 62s - loss: 1.2173 - acc: 0.4843 - val_loss: 1.2438 - val_acc: 0.6471
Epoch 8/20
828/828 [==============================] - 62s - loss: 1.1791 - acc: 0.5217 

In [19]:
labels = np.array(labels)

In [20]:
labels

array([4, 4, 2, 1, 1, 6, 3, 2, 2, 1, 4, 2, 3, 7, 4, 4])

In [14]:

from keras.engine.topology import Merge, merge 
from IPython.display import SVG 
#from keras.utils.visualize_util import model_to_dot



max_sentences = 100
input_dimension = 4800
hidden_state_dim = 500
num_classes = 7

model = Sequential()
model.add(Reshape((1, max_sentences, input_dimension), input_shape=(max_sentences, input_dimension)))
model.add(TimeDistributed(LSTM(hidden_state_dim, input_shape=(max_sentences, input_dimension), return_sequences=True), input_shape=(max_sentences, input_dimension)))
model.add(Reshape((max_sentences,hidden_state_dim), input_shape=(1,max_sentences,hidden_state_dim)))
model.add(TimeDistributed(Dense(hidden_state_dim, activation='tanh', input_shape=(max_sentences,hidden_state_dim))))


alpha = Sequential()
alpha.add(Reshape((1, max_sentences, input_dimension), input_shape=(max_sentences, input_dimension)))
alpha.add(TimeDistributed(LSTM(hidden_state_dim, input_shape=(max_sentences, input_dimension), return_sequences=True), input_shape=(max_sentences, input_dimension)))
alpha.add(Reshape((max_sentences,hidden_state_dim), input_shape=(1,max_sentences,hidden_state_dim)))
alpha.add(TimeDistributed(Dense(hidden_state_dim, activation='tanh', input_shape=(max_sentences,hidden_state_dim))))
alpha.add(TimeDistributed(Dense(hidden_state_dim, activation='softmax', input_shape=(max_sentences,hidden_state_dim), bias=False)))

mergedKerasModel = Sequential()
merged = Merge([model, alpha], mode='dot', dot_axes=1, output_shape=(max_sentences,hidden_state_dim))
mergedKerasModel.add(merged)
mergedKerasModel.add(Lambda(lambda x: K.sum(x, axis=1, keepdims=True), output_shape=lambda s: (s[0], s[2]), input_shape=(max_sentences,hidden_state_dim)))
mergedKerasModel.add(Reshape((hidden_state_dim,)))
mergedKerasModel.add(Dense(num_classes, input_dim=hidden_state_dim, activation='softmax'))

mergedKerasModel.compile(loss='sparse_categorical_crossentropy',optimizer='rmsprop',metrics=['acc'])
print(mergedKerasModel.summary())
#SVG(model_to_dot(mergedKerasModel).create(prog='dot', format='svg'))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
reshape_11 (Reshape)             (None, 1, 100, 4800)  0                                            
____________________________________________________________________________________________________
timedistributed_5 (TimeDistribute(None, 1, 100, 500)   10602000                                     
____________________________________________________________________________________________________
reshape_12 (Reshape)             (None, 100, 500)      0                                            
____________________________________________________________________________________________________
timedistributed_6 (TimeDistribute(None, 100, 500)      250500                                       
___________________________________________________________________________________________

In [15]:
x_train = data_train
y_train = labels_train
x_val = data_test
y_val = labels_test

In [16]:
mergedKerasModel.fit([x_train,x_train], y_train, validation_data=([x_val,x_val], y_val),nb_epoch=20, batch_size=10)

Train on 828 samples, validate on 34 samples
Epoch 1/20
828/828 [==============================] - 127s - loss: 13.7085 - acc: 0.0942 - val_loss: 14.6959 - val_acc: 0.0588
Epoch 2/20
828/828 [==============================] - 125s - loss: 13.8211 - acc: 0.0966 - val_loss: 14.6959 - val_acc: 0.0588
Epoch 3/20
828/828 [==============================] - 125s - loss: 13.1388 - acc: 0.1365 - val_loss: 9.9553 - val_acc: 0.3529
Epoch 4/20
828/828 [==============================] - 128s - loss: 11.9718 - acc: 0.2114 - val_loss: 9.9553 - val_acc: 0.3529
Epoch 5/20
828/828 [==============================] - 128s - loss: 11.9718 - acc: 0.2114 - val_loss: 9.9553 - val_acc: 0.3529
Epoch 6/20
828/828 [==============================] - 127s - loss: 11.9718 - acc: 0.2114 - val_loss: 9.9553 - val_acc: 0.3529
Epoch 7/20
828/828 [==============================] - 129s - loss: 11.9718 - acc: 0.2114 - val_loss: 9.9553 - val_acc: 0.3529
Epoch 8/20
828/828 [==============================] - 127s - loss: 11.9